In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.regularizers import l2
from keras.regularizers import l2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
from tensorflow.keras.callbacks import History
import torch.nn.functional as F


# Data

In [26]:
data = pd.read_csv('train.csv')

In [27]:
X = data.drop('label', axis=1).values
y = data['label'].values

In [28]:
X_train, X_tes, y_train, y_tes = train_test_split(X, y, test_size=0.2, random_state=42)


In [29]:
X_test, X_val, y_test, y_val = train_test_split(X_tes, y_tes, test_size=0.5, random_state=42)


In [30]:
X_train_r = X_train.reshape(-1, 1, 32, 32) / 255.0
X_val_r = X_val.reshape(-1, 1, 32, 32) / 255.0
X_test_r = X_test.reshape(-1, 1, 32, 32) / 255.0 #normalizing

In [35]:
train_tensor = TensorDataset(torch.Tensor(X_train_r), torch.Tensor(y_train).long())
val_tensor = TensorDataset(torch.Tensor(X_val_r), torch.Tensor(y_val).long())
test_tensor = TensorDataset(torch.Tensor(X_test_r), torch.Tensor(y_test).long())

#train test val tensor

In [36]:
train_loader = DataLoader(train_tensor, batch_size=64, shuffle=True)
val_loader = DataLoader(val_tensor, batch_size=64, shuffle=False)
test_loader = DataLoader(test_tensor, batch_size=64, shuffle=False)

# Final model

In [10]:
class CustomCNN(nn.Module):
    def __init__(self, layers):
        super(CustomCNN, self).__init__()
        self.layers = nn.ModuleList()
        input_channels = 1  

        for output_channels in layers:
            self.layers.append(nn.Conv2d(input_channels, output_channels, kernel_size=3, padding=1))
            self.layers.append(nn.ReLU())
            self.layers.append(nn.MaxPool2d(2))
            input_channels = output_channels

        # classification layer
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_channels * (32 // 2**len(layers))**2, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = self.classifier(x)
        return x


In [11]:
model = CustomCNN([64, 128])
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam with best learning rate
criterion = nn.CrossEntropyLoss()


In [12]:
def train_and_validate(model, train_loader, val_loader, optimizer, criterion, epochs):
    model.train()
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{epochs} training complete.')

    
    model.eval()
    with torch.no_grad():
        val_loss = 0
        correct = 0
        total = 0
        for inputs, labels in val_loader:
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        print(f'Validation Accuracy: {val_accuracy:.2f}%')

    return model

In [13]:
final_model = train_and_validate(model, train_loader, val_loader, optimizer, criterion, epochs=10)


Epoch 1/10 training complete.
Epoch 2/10 training complete.
Epoch 3/10 training complete.
Epoch 4/10 training complete.
Epoch 5/10 training complete.
Epoch 6/10 training complete.
Epoch 7/10 training complete.
Epoch 8/10 training complete.
Epoch 9/10 training complete.
Epoch 10/10 training complete.
Validation Accuracy: 90.21%


In [16]:
# saving trained model
torch.save(final_model.state_dict(), 'final_best_model.pth')

# Test data

In [17]:
model = CustomCNN([64, 128])

#weights
model.load_state_dict(torch.load('final_best_model.pth'))

# evaluation
model.eval()

CustomCNN(
  (layers): ModuleList(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=8192, out_features=128, bias=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=10, bias=True)
  )
)

In [37]:
def evaluate_model(model, data_loader):
    model.eval()  # Перевод модели в режим оценки
    total = 0
    correct = 0
    total_loss = 0.0
    criterion = torch.nn.CrossEntropyLoss()  # Использование CrossEntropyLoss для классификации

    with torch.no_grad():  # Отключение градиентов для ускорения вычислений
        for inputs, labels in data_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / total
    accuracy = 100 * correct / total
    return avg_loss, accuracy

In [38]:
test_loss, test_accuracy = evaluate_model(model, test_loader)

In [43]:
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.2f}%')

Test Loss: 0.3138
Test Accuracy: 90.25%


Model's perfomance on test data is 90%

# Evaluation

In [41]:
test_data = pd.read_csv('evaluate.csv')
X_test = test_data.drop('ID', axis=1).values
X_test = X_test.reshape(-1, 1, 32, 32) / 255.0  
test_ids = test_data['ID']

# to tensor pytorch
test_tensor = TensorDataset(torch.Tensor(X_test))

# DataLoader
test_loader = DataLoader(test_tensor, batch_size=64, shuffle=False)

In [42]:
model = CustomCNN([64, 128])  
model.load_state_dict(torch.load('final_best_model.pth'))
model.eval()

# collecting predictions
predictions = []
with torch.no_grad():
    for inputs in test_loader:
        outputs = model(inputs[0])
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.tolist())

In [44]:
results_df = pd.DataFrame({
    'ID': test_ids,
    'label': predictions
})

#df to csv
results_df.to_csv('results.csv', index=False)